# Trend 趋势指示器

趋势指示器提供了一个值及其最近趋势的可视化表示。它支持向图表组件流式传输数据，使得能够对某个值的最近趋势提供高性能的实时更新。

底层实现为`panel.widgets.Trend`，参数基本一致，参考文档：https://panel.holoviz.org/reference/widgets/Trend.html


In [1]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

最简单的`Trend`只需要提供带有x和y值的`data`，可以声明为字典或`pandas.DataFrame`。`value`和`value_change`值将从数据中自动计算：


In [9]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnTrend 
    name="Price" 
    :data="data" 
    :width="200" 
    :height="200" 
  />
</template>

<script lang='py'>
import numpy as np

np.random.seed(8)
data = {'x': np.arange(50), 'y': np.random.randn(50).cumsum()}
</script>

{"vue": "<template>\n  <PnTrend \n    name=\"Price\" \n    :data=\"data\" \n    :width=\"200\" \n    :height=\"200\" \n  />\n</template>\n\n<script lang='py'>\nimport numpy as np\n\nnp.random.seed(8)\ndata = {'x': np.arange(50), 'y': np.random.randn(50).cumsum()}\n</script>\n", "setup": ""}



## 数据流式传输

`Trend`指示器还提供了一个方便的方法来流式传输新数据，支持`rollover`参数来限制显示的数据量。我们将使用`setInterval`来定期更新图表：


In [20]:
%%vuepy_run --plugins vpanel --show-code
<template>
 <PnCol>
  <PnTrend 
    ref="trend_ref"
    name="Price" 
    :data="data" 
    :width="200" 
    :height="200" 
  />
  <PnButton name="Start Streaming" @click="start()" />
 </PnCol>
</template>

<script lang='py'>
import numpy as np
import panel as pn
from vuepy import ref

data = {'x': np.arange(10), 'y': np.random.randn(10).cumsum()}
interval_id = None
trend_ref = ref(None)

def stream_data():
    trend = trend_ref.value.unwrap()
    trend.stream({
        'x': [trend.data['x'][-1]+1],
        'y': [trend.data['y'][-1]+np.random.randn()]
    },  rollover=50)
    

def start():
    pn.state.add_periodic_callback(stream_data, period=250, count=15);
</script>

{"vue": "<template>\n <PnCol>\n  <PnTrend \n    ref=\"trend_ref\"\n    name=\"Price\" \n    :data=\"data\" \n    :width=\"200\" \n    :height=\"200\" \n  />\n  <PnButton name=\"Start Streaming\" @click=\"start()\" />\n </PnCol>\n</template>\n\n<script lang='py'>\nimport numpy as np\nimport panel as pn\nfrom vuepy import ref\n\ndata = {'x': np.arange(10), 'y': np.random.randn(10).cumsum()}\ninterval_id = None\ntrend_ref = ref(None)\n\ndef stream_data():\n    trend = trend_ref.value.unwrap()\n    trend.stream({\n        'x': [trend.data['x'][-1]+1],\n        'y': [trend.data['y'][-1]+np.random.randn()]\n    },  rollover=50)\n    \n\ndef start():\n    pn.state.add_periodic_callback(stream_data, period=250, count=15);\n</script>\n", "setup": ""}



## 图表类型

除了默认的`plot_type`外，流指示器还支持其他几种选项：


In [12]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnRow>
    <PnTrend 
      v-for="plot_type in plot_types"
      :name="plot_type" 
      :data="data" 
      :plot_type="plot_type"
      :width="150" 
      :height="150" 
    />
  </PnRow>
</template>

<script lang='py'>
import numpy as np

data = {'x': np.arange(50), 'y': np.random.randn(50).cumsum()}
plot_types = ['line', 'bar', 'step', 'area']
</script>

{"vue": "<template>\n  <PnRow>\n    <PnTrend \n      v-for=\"plot_type in plot_types\"\n      :name=\"plot_type\" \n      :data=\"data\" \n      :plot_type=\"plot_type\"\n      :width=\"150\" \n      :height=\"150\" \n    />\n  </PnRow>\n</template>\n\n<script lang='py'>\nimport numpy as np\n\ndata = {'x': np.arange(50), 'y': np.random.randn(50).cumsum()}\nplot_types = ['line', 'bar', 'step', 'area']\n</script>\n", "setup": ""}



## API

### 属性

| 属性名        | 说明                      | 类型                                                           | 默认值 |
| ------------ | ------------------------ | --------------------------------------------------------------| ------- |
| data         | 图表数据                   | ^[object\|object[]] 字典或pandas DataFrame形式的数据            | —      |
| layout       | 指示器的布局               | ^[string] 可选值: 'column', 'row'                             | column  |
| plot_x       | 数据中对应图表x值的列       | ^[string]                                                      | y      |
| plot_y       | 数据中对应图表y值的列       | ^[string]                                                      | x      |
| plot_color   | 图表中使用的颜色            | ^[string]                                                      | #428bca |
| plot_type    | 绘制图表数据的图表类型      | ^[string] 可选值: 'line', 'bar', 'step', 'area'                | bar     |
| pos_color    | 用于指示正向变化的颜色      | ^[string]                                                      | #5cb85c |
| neg_color    | 用于指示负向变化的颜色      | ^[string]                                                      | #d9534f |
| value        | 要显示的主值               | ^[number\|string] 数字或"auto"                                 | auto    |
| value_change | 值变化表示为分数           | ^[number\|string] 数字或"auto"                                 | auto    |
| disabled     | 是否禁用                  | ^[boolean]                                                     | false   |

### Events

| 事件名 | 说明                  | 类型                                   |
| ---   | ---                  | ---                                    |
| change | 当值变化时触发的事件   | ^[Callable]`(event: dict) -> None`     |

### Slots

| 插槽名   | 说明               |
| ---     | ---               |

### 方法

| 名称      | 说明                             | 参数                                                  |
| -------- | -------------------------------- | ----------------------------------------------------|
| stream   | 向图表流式传输新数据，支持限制显示的数据量 | data: 要添加的新数据, rollover: 保留的最大数据点数量    |


In [14]:
##ignore
import numpy as np
import pandas as pd
import panel as pn

pn.extension()

data = {'x': np.arange(50), 'y': np.random.randn(50).cumsum()}

trend = pn.widgets.Trend(
    name='Price', data=data, width=200, height=200
)
pn.Row(trend.controls(), trend)

Row
    [0] Tabs
        [0] WidgetBox(margin=(5, 10), name='Controls')
            [0] StaticText(value='<b>Controls</b>')
            [1] LiteralInput(description='The primary v..., name='Value', serializer='json', value='auto')
            [2] Checkbox(name='Disabled')
            [3] Select(name='Layout', options=OrderedDict([('column', ...]), value='column')
            [4] TextInput(description='The name of t..., name='Plot x', value='x')
            [5] TextInput(description='The name of t..., name='Plot y', value='y')
            [6] TextInput(description='The color to u..., name='Plot color', value='#428bca')
            [7] Select(description='The plot type t..., name='Plot type', options=OrderedDict([('line', ...]), value='bar')
            [8] TextInput(description='The color used t..., name='Pos color', value='#5cb85c')
            [9] TextInput(description='The color used t..., name='Neg color', value='#d9534f')
            [10] LiteralInput(description='A secondary v..., name='Value change', serializer='json', value='auto')
        [1] WidgetBox(margin=(5, 10), name='Layout')
            [0] StaticText(value='<b>Layout</b>')
            [1] TextInput(description='The name or a..., name='Name', value='Price')
            [2] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
            [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
            [4] ListInput(description='CSS classes t..., name='Css classes', serializer='json', type=<class 'list'>)
            [5] IntInput(description='The height of the compone..., name='Height', start=0, value=200)
            [6] IntInput(description='Minimal width o..., name='Min width', start=0)
            [7] IntInput(description='Minimal height o..., name='Min height', start=0)
            [8] IntInput(description='Maximum width o..., name='Max width', start=0)
            [9] IntInput(description='Maximum height o..., name='Max height', start=0)
            [10] LiteralInput(description='Allows to create addition..., name='Margin', serializer='json', value=(5, 10))
            [11] DictInput(description='Dictionary of C..., name='Styles', serializer='json', type=<class 'dict'>)
            [12] ListInput(description='List of arbitrary t..., name='Tags', serializer='json', type=<class 'list'>)
            [13] IntInput(description='The width of the componen..., name='Width', start=0, value=200)
            [14] Select(description='Describes how the compone..., name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [15] Select(description='Describes how the compone..., name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [16] Select(description='How the component should ..., name='Sizing mode', options=OrderedDict([('fixed', ...]), value='fixed')
            [17] Checkbox(name='Visible', value=True)
    [1] Trend(data={'x': array([ 0, ...}, height=200, name='Price', sizing_mode='fixed', width=200)